In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.

In [ ]:
import json
train_path = "/content/drive/MyDrive/IDL-Project/WebQA_data_first_release/WebQA_train_val.json"
q = json.load(open(train_path,'r'))

In [ ]:
import torch
import torch.nn as nn
import json
import os
# import clip
from torch.utils.data import DataLoader, Dataset
try:
  from transformers import BertTokenizer
except:
  %pip install transformers
  from transformers import BertTokenizer
import pickle
from collections import defaultdict

MODALS = ['img_posFacts','img_negFacts', 'img_posFacts','img_negFacts']

class EmbedTunerDataset(Dataset):
  def __init__(self,  img_folder, json_data, imgid_map_path, mode='train'):
    
    self.imgid = pickle.load(open(imgid_map_path, "rb")) 
    self.data = json_data
    self.mode = mode
    self.img_folder = img_folder
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    self.instances = list(self.data.keys())
    self.Qs = [self.data[instance]['Q'].replace('"', "") for instance in self.instances]
    # self.lens = [(Q['input_ids'] == 0).nonzero(as_tuple=False)[0][1].numpy() for Q in self.Qs]


  
  def __len__(self):
    return len(self.instances)
  
  def __getitem__(self, idx):

    source_feats = defaultdict(list)
    sources = self.data[self.instances[idx]] # all sources of this idx
    pos_neg_tag = None
    for mode in MODALS:
      if len(sources[mode]): # if this type of source exists
        for source in sources[mode]:
          
          if 'img' in mode: # if it is img load feats
            image_id = self.imgid[int(source['image_id'])]
            feats = os.path.join(self.img_folder, \
                                 "{}/{}/{}.pkl".format(self.mode,\
                                                       (image_id%10000000)//1000,\
                                                       image_id))
            
            with open(feats,'rb') as f:
              feats = pickle.load(f)
              img = feats['fc1_features'].detach().cpu().float()
              cls_label = feats['cls_features'].detach().cpu().float()
              feats = torch.cat([img, cls_label], dim=1).mean(dim=0)
            
          else: # it is a txt source
            feats = source['fact'].replace('"', "")

          pos_neg_tag = 1. if 'pos' in mode else -1
          source_feats[mode].append((feats,pos_neg_tag))

    return self.Qs[idx], source_feats
  
  @staticmethod
  def collate_fn(batch):
    Qs = [b[0] for b in batch]
    source_feats = [b[1] for b in batch] # these are dicts with each type of source

    new_batches = []
    qs, srcs, src_types, tags = [], [], [], []
    for q, source_feat in zip(Qs, source_feats):
      for key, val in source_feat.items():
        src_type = 'img' if 'img' in key else 'txt'
        for src, tag in val:
          # datapt = (q, src, src_type, tag) # q_tokens, img or txt feats, type
          # new_batches.append(datapt)
          qs.append(q), srcs.append(src), src_types.append(src_type), tags.append(tag)

    return (qs, srcs, src_types, tags)



# test_path = "/content/drive/MyDrive/18-786-Intro-DL/IDL-Project/WebQA_data_first_release/WebQA_test.json"
# q =  json.load(open(test_path,'r'))

img_folder = "/content/drive/MyDrive/IDL-Project/WebQA 101 Features"# path to feats folder
imgid_map_path = "/content/drive/MyDrive/IDL-Project/WebQA 101 Features/image_id_map_0328.pkl"#path to map file

val_pts = {k:v for k,v in q.items() if v['split'] == 'val'}
train_pts = {k:v for k,v in q.items() if v['split'] == 'train'}

train_loader = DataLoader(EmbedTunerDataset(img_folder, train_pts, imgid_map_path, mode='train'),batch_size=5,shuffle=True, collate_fn=EmbedTunerDataset.collate_fn)
val_loader = DataLoader(EmbedTunerDataset(img_folder, val_pts, imgid_map_path, mode='val'),batch_size=5,shuffle=False, collate_fn=EmbedTunerDataset.collate_fn)


In [ ]:
next(iter(train_loader))[1][0]

tensor([ 1.3101, -2.8742, -2.4465,  ...,  1.3623, -0.7533, 13.8441])

In [ ]:
! pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-252_z3rp
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-252_z3rp
     |████████████████████████████████| 53 kB 1.7 MB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369389 sha256=5a27ddb57dcdc444058e320526336ab3a59f9f0bccf00a9ac5a5612e5b5ee234
  Stored in directory: /tmp/pip-ephem-wheel-cache-w6cw13xk/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [ ]:
from torch import nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import clip
import numpy as np

class Clip_Model(nn.Module):

    def __init__(self):

        super(Clip_Model, self).__init__()
        
        # print(configuration.hidden_size)
        self.model, self.preprocess = clip.load("ViT-B/32") 
        self.linear_img = nn.Linear(3649, 512)
        self.linear_txt = nn.Linear(512, 512)

    def forward(self, qs, feats, ftype):
        question_tokens   = clip.tokenize(qs, truncate=True).to("cuda")
        question_features = self.model.encode_text(question_tokens).float()
        ftype = np.array(ftype)
        img_idx = np.where(ftype=='img')[0]
        txt_idx = np.where(ftype=='txt')[0]
        feats_img = [feats[i] for i in img_idx]
        feats_txt = [feats[i] for i in txt_idx]
        features = []
        question_features_ordered = []
        # print(f'feats_img {feats_img}')
        # print(f'feats_img {torch.stack(feats_img, dim=0).shape}')
        if len(txt_idx):
          knowledge_source_tokens = clip.tokenize(feats_txt, truncate=True).to("cuda")
          knowledge_source_feats = self.model.encode_text(knowledge_source_tokens).float()
          features.extend(self.linear_txt(knowledge_source_feats))
          question_features_ordered.extend(question_features[txt_idx])
        if len(img_idx):
          features.extend(self.linear_img(torch.stack(feats_img, dim=0)))
          question_features_ordered.extend(question_features[img_idx])
        
        # print(f'question_features_ordered {torch.cat(question_features_ordered, dim=0).shape}')
        # print(f'features {torch.cat(features, dim=0).shape}')
        return torch.stack(question_features_ordered, dim=0), torch.stack(features, dim=0)
        

In [ ]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def customLoss(q_feat, src_feat, tag):
  # loss = tag*(torch.norm(q_feat-src_feat, dim=1)) + (1-tag)*(-torch.norm(q_feat-src_feat, dim=1))
  loss = torch.nn.CosineEmbeddingLoss(reduction="sum")(q_feat, src_feat, tag)
  return loss

def train(model, train_dataloader, epochs, optimizer):
  model.train()

  total_acc_train = 0
  total_loss_train = 0

  criterion = nn.CrossEntropyLoss()


  for epoch in range(epochs):
    num_egs = 0.0
    model.train()
    model.model.requires_grad_(False)

    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, position=0, leave=False, desc='Train')

    for batch_idx, data in enumerate(train_loader):
      # print(len(data))
      (qs, feats, featType, tag) = data
      num_egs += len(qs)
      
      # print(input_id)

      question_features, features = model(qs, feats, featType)
      
      batch_loss = customLoss(question_features.to(device), features.to(device), torch.tensor(tag).to(device))
      total_loss_train += batch_loss.item()
      

      model.zero_grad()
      batch_loss.backward()
      optimizer.step()

      batch_bar.set_postfix(loss="{:.06f}".format(total_loss_train/num_egs))

      batch_bar.update()
    

    batch_bar.close()

    print(f'Train loss is total_acc_val {total_loss_train/num_egs}')

    # total_loss_val = 0.0
    # total_acc_val = 0.0

    # with torch.no_grad():
    #   num_egs = 0.0
    #   model.eval()
    #   batch_bar = tqdm(total=len(val_dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val')


    #   for batch_idx, (qs, length, modality) in enumerate(val_dataloader):
    #       num_egs += modality.shape[0]
    #       modality = modality.to(device)
    #       mask = qs['attention_mask'].to(device)
    #       input_id = qs['input_ids'].squeeze(1).to(device)

    #       output = model(input_id, mask)

    #       batch_loss = criterion(output, modality)
    #       total_loss_val += batch_loss.item()
          
    #       acc = (output.argmax(dim=1) == modality).sum().item()
    #       total_acc_val += acc

    #       batch_bar.set_postfix(acc="{:.06f}".format(total_acc_val/num_egs))

    #       batch_bar.update()

    #   batch_bar.close()

    #   save(model, args, epoch, optimizer, None)
    
    #   print(f'Val accuracy is total_acc_val {total_acc_val/num_egs}')


In [ ]:
model = Clip_Model()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
epochs = 10
train(model, train_loader, epochs, optimizer)

Train:  23%|██▎       | 1655/7354 [16:56<1:00:11,  1.58it/s, loss=0.054214]

KeyboardInterrupt: ignored

In [ ]:
import clip

model, preprocess = clip.load("ViT-B/32")
question_tokens   = clip.tokenize(["question"]).to("cuda")
question_features = model.encode_text(question_tokens).float()

In [ ]:
question_features.shape

torch.Size([1, 512])

In [ ]:
next(iter(train_loader))[0]

('Were Larry Glick and Laurence Gluck born in the same year?',
 tensor([ 1.3101, -2.8742, -2.4465,  ...,  1.3623, -0.7533, 13.8441]),
 'img',
 0.0)

In [ ]:
! ls "/content/drive/MyDrive/IDL-Project/WebQA 101 Features/Train/" | wc -l

250


In [ ]:
!7za x /content/drive/MyDrive/IDL-Project/Web_QA_feats_subset


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/IDL-Project/                                               0M 1 Scan /content/drive/MyDrive/IDL-Project/Web_QA_feats_subset/                                                                   1 folder, 1 file, 564540634 bytes (539 MiB)

Extracting archive: /content/drive/MyDrive/IDL-Project/Web_QA_feats_subset/140.7z
--
Path = /content/drive/MyDrive/IDL-Project/Web_QA_feats_subset/140.7z
Type = 7z
Physical Size = 564540634
Headers Size = 4194
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 1

In [ ]:
! pwd

/content


In [ ]:
with open('/content/140/40140580.pkl','rb') as f:
  x = pickle.load(f)

In [ ]:
x['fc1_features'].shape

torch.Size([100, 2048])

In [ ]:
x['cls_features'].shape

torch.Size([100, 1601])

In [ ]:
x['pred_boxes'].shape

torch.Size([100, 4])

In [ ]:
import torch.nn.functional as F

features = x

img = features['fc1_features'].detach().cpu().float()
cls_label = features['cls_features'].detach().cpu().float()
feats = torch.cat([img, cls_label], dim=1).mean(dim=0)
# vis_pe = features['pred_boxes'].detach().cpu()

# Lazy normalization of the coordinates
# w_est = torch.max(vis_pe[:, [0, 2]])*1.+1e-5
# h_est = torch.max(vis_pe[:, [1, 3]])*1.+1e-5
# vis_pe[:, [0, 2]] /= w_est
# vis_pe[:, [1, 3]] /= h_est

# rel_area = (vis_pe[:, 3]-vis_pe[:, 1])*(vis_pe[:, 2]-vis_pe[:, 0])
# rel_area.clamp_(0)

# vis_pe = torch.cat((vis_pe[:, :4], rel_area.view(-1, 1), features['scores'].detach().cpu().view(-1, 1)), -1)
# normalized_coord = F.normalize(vis_pe.data[:, :5] - 0.5, dim=-1)
# vis_pe = torch.cat((F.layer_norm(vis_pe, [6]), F.layer_norm(cls_label, [1601])), dim=-1)

# assert img.size(0) == vis_pe.size(0), "img features and vis_pe should have the same token length!"
# vis_pad = torch.zeros((self.max_len_img_cxt - img.size(0), img.size(-1)))
# img = torch.cat((img, vis_pad), dim=0) 
# pe_pad = torch.zeros((self.max_len_img_cxt - vis_pe.size(0), vis_pe.size(-1)))
# vis_pe = torch.cat((vis_pe, pe_pad), dim=0)
# assert vis_pe.size(0) == self.max_len_img_cxt
# assert img.size(0) == self.max_len_img_cxt
# input_ids_list.append(torch.tensor(input_ids))
# segment_ids_list.append(torch.tensor(segment_ids))
# input_mask_list.append(input_mask)
# if not self.use_img_content: 
#     img = torch.zeros_like(img).float()
#     vis_pe = torch.zeros_like(vis_pe).float()
# img_list.append(img)
# vis_pe_list.append(vis_pe)

In [ ]:
feats.shape

torch.Size([3649])

In [ ]:
def customLoss(q_feat, src_feat, tag, eps):
  loss = tag*(torch.norm(q_feat-src_feat, dim=1)) + (1-tag)*torch.max(0,eps-torch.norm(q_feat-src_feat, dim=1))
  return torch.mean(loss)

In [ ]:
x = [1,2,3,4,5]

x[2,3]

TypeError: ignored

In [ ]:
x = ['img', 'img', 'img']

In [ ]:
np.where(x=='img')[0]

array([0, 1, 2])